In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,726 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/restricted amd64 Packages [2,412 kB]
Get:12 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,052 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubunt

In [4]:
# JAVA_HOME 환경변수 설정하기

%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
# 분석하기

import csv
import json
import os
from konlpy.tag import Okt
import csv

# Okt 객체 생성
okt = Okt()

# data 폴더 경로 설정
data_folder = '/content/drive/MyDrive/csv'

# data 폴더 내의 파일 리스트 가져오기
file_list = os.listdir(data_folder)

# 감성 사전 파일 경로
sentiment_dict_file = '/content/drive/MyDrive/Colab Notebooks/sentiment_dictionary_saved.json'

# 감성 사전 로드
with open(sentiment_dict_file, 'r', encoding='utf-8') as file:
    sentiment_dictionary = json.load(file)

# 파일 리스트 출력
for i in range(len(file_list)):
    file_list[i] = os.path.join(data_folder, file_list[i])
file_list.sort()

In [19]:
results = []
for i in range(len(file_list)):
    with open(file_list[i], 'r', encoding='utf-8') as file:
        file_name_without_extension = os.path.split(file_list[i])[1]
        # print(file_name_without_extension)

        # 형태소 분석된 결과를 저장할 리스트와 변수 초기화
        analyzed_reviews = []

        # 리더 
        reader = file.readlines()

        # 리뷰들을 형태소 분석하여 리스트에 추가
        for row in reader:
            review = row  # 리뷰 컬럼
            morphs = okt.morphs(review)
            analyzed_reviews.append(morphs)

        # 형태소 분석된 결과를 활용한 감성 점수 계산
        review_Num = len(analyzed_reviews)
        sentiment_score = 0  # 초기 감성 점수
        neg_freq = 0 # 나쁜 말이 나온 수
        neg_val = 0
        pos_freq = 0 # 좋은 말이 나온 수
        pos_val = 0
        review_scores = []
        # 모든 분석된 리뷰들에 대해
        for review in analyzed_reviews:
            # 각 리뷰의 모든 형태소들에 대해
            for morph in review:
                review_score = 0
                # 만약 형태소가 사전에 있다면
                if morph in sentiment_dictionary:
                    
                    # 그 극성을 저장하기
                    tmp = sentiment_dictionary[morph]
                    # 나쁜 형태소가 나온 빈도
                    if tmp < 0:
                        neg_freq += 1
                        neg_val += tmp
                    # 좋은 형태소가 나온 빈도
                    elif tmp > 0:
                        pos_freq += 1
                        pos_val += tmp
                    review_score += tmp
                    sentiment_score += tmp

                if review_score != 0:
                    review_scores.append(review_score)
        # print(review_scores)
        # print('총 감성 점수:', sentiment_score)
        # print("pos: ", pos)
        # print("neg: ", neg)
        results.append((file_name_without_extension,sentiment_score, pos_freq, pos_val, neg_freq, neg_val, review_Num))

In [20]:
import csv


# CSV 파일 경로
csv_file = '/content/drive/MyDrive/Colab Notebooks/results.csv'

# CSV 파일에 데이터 쓰기
with open(csv_file, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['file_name_without_extension','sentiment_score', 'pos_freq', 'pos_val', 'neg_freq', 'neg_val', 'review_Num'])  # 헤더 작성
    writer.writerows(results)  # 데이터 작성